<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install elephas
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 41.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=77d74759c73d114b09ad2d1c3459ff7ec8c2d1ec1cd2ebc9964026e22ad78996
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 4.1 MB 28.5 MB/s 
     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 48.7 MB/s 
     |████████████████████████████████| 462 kB 52.8 MB/s 
  Created wheel for elephas: filename=elephas-3.1.0-py3-none-any.whl size=26259 sha256=0779cf0f27d3d5fff9186104346c9daf1a07727082aadba15c25ffb3e067a53b
  Stored in directory: /root/.cache/pip/wheels/33/e4/e1/56dda8be927bb0e9971cd7ddf3fc1b17ce78db56268b1f867f
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=dc44

In [ ]:
import pyspark
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics

# Keras / Deep Learning
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from tensorflow.keras.optimizers import Adam

# Elephas for Deep Learning on Spark
from elephas.ml_model import ElephasEstimator

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
from google.colab import files
import os
import zipfile
import findspark
from pyspark.sql.functions import col
from petastorm import TransformSpec
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import numpy as np

#from my model
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import pickle
import shutil
import random
import skimage.io as io
from copy import deepcopy



In [ ]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
conf = SparkConf().setAppName('Face_Comics_recognition').setMaster('local[6]')
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)

ValueError: ignored

In [ ]:
os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")

!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

 99% 2.16G/2.18G [00:17<00:00, 163MB/s]
100% 2.18G/2.18G [00:17<00:00, 133MB/s]


In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"


findspark.init(findspark.find())
spark = SparkSession.builder.master("local").appName("Colab").getOrCreate()
spark

In [ ]:
df_comics = spark.read.format("image").load(".//Face_Comics_data//comics").withColumn("label", lit(int(1)))
df_faces = spark.read.format("image").load(".//Face_Comics_data//faces").withColumn("label", lit(int(0)))

#df_comics.select("image.height","image.width","image.nChannels", "image.mode", "image.data").show(1)
#df_comics.show(5)
df_comics = df_comics.select(col("image.data"),col("label"))
df_faces = df_faces.select(col("image.data"),col("label"))
df_comics.show(5)
df_faces.show(5)

+--------------------+-----+
|                data|label|
+--------------------+-----+
|[52 58 6F 40 46 5...|    1|
|[AA A3 94 A9 A2 9...|    1|
|[64 5C 55 61 59 5...|    1|
|[A6 9E 91 A9 A1 9...|    1|
|[9E 93 8B 9A 8F 8...|    1|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|                data|label|
+--------------------+-----+
|[8E AE 95 81 A4 8...|    0|
|[54 91 77 3A 7A 5...|    0|
|[46 7E 97 7D B5 C...|    0|
|[DC F0 DD E5 FE E...|    0|
|[8D B6 AF 87 B0 A...|    0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
#Read images and Create training & test DataFrames for transfer learning
#Shuffle Data
df_comics = df_comics.orderBy(rand())
df_faces = df_faces.orderBy(rand())

comics_train, comics_validation, comics_test = df_comics.randomSplit([0.6, 0.2,0.1], seed = 1234)
print(comics_train)
faces_train, faces_validation, faces_test = df_faces.randomSplit([0.6, 0.2,0.1], seed = 1234)
print(faces_train)

#dataframe for training a classification model
train_df = faces_train.unionAll(comics_train)

#dataframe for validating the classification model
val_df = faces_validation.unionAll(comics_validation)

#dataframe for testing the classification model
test_df = faces_test.unionAll(comics_test)

num_classes = test_df.select("label").distinct().count()
train_df = train_df.repartition(2)
print("number of classes", num_classes)

KeyboardInterrupt: ignored

In [ ]:
def preprocess(content):
  image = image.Open(io.BytesIO(content)).resize([350,350])
  image_array = keras.preprocessing.image.img_to_array(image)
  return tf.keras.applications.mobilnte_v2.preprocess_input(image_array)

def transform_batch(pd_batch):
  pd_batch['features'] = pd_batch['content'].map(lambda x: preprocess(x))
  pd_batch = pd_batch.drop(labels='content', axis = 1)
  return pd_batch

IMG_SHAPE = (350, 350, 3)
EPOCHS = 15
BATCH_SIZE = 32
IMG_SIZE = (350, 350)

transform_spec_fn = TransformSpec(
    transform_batch,
    edit_fields = [('features', np.float32, IMG_SHAPE, False)],
    selected_fields = ['features','label_index']
)


In [ ]:
import pandas as pd
pd_batch0 = train_df.limit(5).toPandas()
pd_batch0

KeyboardInterrupt: ignored

In [ ]:
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, ".//staging")

converter_train = make_spark_converter(train_df)
converter_val = make_spark_converter(val_df)

In [ ]:
#def training_and_validation()
train_dataset = converter_train.make_tf_dataset(transform_spec=transform_spec_fn, 
                                batch_size = BATCH_SIZE) 

val_dataset = converter_val.make_tf_dataset(transform_spec=transform_spec_fn, 
                                batch_size = BATCH_SIZE) 


train_dataset = train_dataset.map(lambda x: (x.features, x.label_index))
val_dataset = val_dataset.map(lambda x: (x.features, x.label_index))
train_dataset

DataFrame[data: binary, label: int]


In [ ]:
#data augmentation


def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (180-x)
  else:
    x
  return x

###Randim invert
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))


class RandomInvert(layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)


###Kernel Inizializer Sobel_x
def hedge_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [-5, -4, 0, 4, 5], 
            [-8, -10, 0, 10, 8], 
            [-10, -20, 0, 20, 10], 
            [-8, -10, 0, 10, 8], 
            [-5, -4, 0, 4, 5]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x


def vertical_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 0, -1], 
            [1, 0, -1], 
            [1, 0, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

def horizontal_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 1, 1], 
            [0, 0, 0], 
            [-1, -1, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

In [ ]:
###Create CNN
#Initialize the CNN
cnn = tf.keras.models.Sequential()
    
cnn.add(tf.keras.Sequential([
	layers.RandomFlip("horizontal_and_vertical"),
	layers.RandomRotation(0.3),
    layers.RandomContrast(0.5, seed=None),
    RandomInvert(),
    layers.RandomZoom(height_factor=(-0.2, +0.3)),
    layers.RandomTranslation(height_factor=(-0.2, +0.3),width_factor=(-0.2, +0.3)),
    layers.Rescaling(1./255, offset= -1)
    ]))

DROPOUT
#Introduce the Convolution layer with Kernel_initializer=Sobel_x
#Convolution
cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, kernel_initializer = hedge_detector, strides=2, activation='relu',  input_shape = [350,350,3]))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Second Convolutional Layer
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
#Third Convolutional Layer
#cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, activation = 'relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))
#Flattening
cnn.add(tf.keras.layers.Flatten())
#Full Connection
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
#DROPOUTS ok(0.1)
cnn.add(tf.keras.layers.Dropout(0.1))
#Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
###Training CNN
#Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the train_dataset and validate it on the validation_dataset
model_fit = cnn.fit(x = train_dataset, validation_data = val_dataset, epochs = EPOCHS)

cnn.summary()


In [ ]:
#There are two variable objects. Divided between around 2.5 million of MobilNet parameters which 
#are frozen, and 1.2 thousend of trainable parameter in the Dense layer
len(cnn.trainable_variables)

acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
loss_ = model_fit.history['loss']
val_loss_ = model_fit.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color = 'gray',linestyle='dashed')
plt.plot(val_acc, label='Validation Accuracy', color = 'black')
plt.ylim([0.8, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss_, label='Training Loss', color = 'gray',linestyle='dashed')
plt.plot(val_loss_, label='Validation Loss', color = 'black')
plt.ylim([0, 1.0])
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
"""#pesca dalla folder scaricata, non dal flusso spark
data_dir= os.environ["data_source"]

os.listdir(data_dir)

tf.random.set_seed(123456)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    shuffle=True,
    seed=123456,
    image_size= (160,160),
    batch_size=32)


class_names = train_data.select("label_index").distinct().count()
print(class_names)
"""